In [1]:
import sqlite3
import time

conn = sqlite3.connect("../feature_machine_learning_test/bitcoin_db/index_620000.db")
cur = conn.cursor()
tmp_conn = sqlite3.connect("../feature_machine_learning_test/bitcoin_db/index_620000.db")
tmp_cur = tmp_conn.cursor()

# database의 table확인하기
cur.execute('SELECT name from sqlite_master where type= "table"')
print(cur.fetchall())

[('BlkID',), ('TxID',), ('AddrID',), ('Meta',), ('TxIn',), ('TxOut',)]


In [2]:
import pandas as pd

path = './ChainAnalysisNamedClusters.csv'
category_addr = pd.read_csv(path)
category_addr = category_addr[['Category', 'RootAddress']]
category_addr.head()

,Category,RootAddress
0,darknet market,17Bccs6eJAKsXHeopcH5FCt7o1FdCk1Xw7
1,gambling,1rH4vGpf5LEgCvUxXgRycGrQ5ahsxXB2F
2,scam,1Kd9zuLiJ5mbz6YD7a8nQiiAXvPuveUyRL
3,exchange,14zRXp2AZ8yxXzPXJ5gAirmgAxJdUZiVut
4,mining pool,1F1xcRt8H8Wa623KqmkEontwAAVqDSAWCV


In [3]:
select_category_addr = category_addr[category_addr['Category'].isin(['darknet market', 'exchange', 'gambling', 'high risk exchange', 'hosted wallet', 'merchant services', 'mining pool', 'mixing', 'ransomware', 'scam', 'stolen funds', 'other'])]
select_category_addr.head()

,Category,RootAddress
0,darknet market,17Bccs6eJAKsXHeopcH5FCt7o1FdCk1Xw7
1,gambling,1rH4vGpf5LEgCvUxXgRycGrQ5ahsxXB2F
2,scam,1Kd9zuLiJ5mbz6YD7a8nQiiAXvPuveUyRL
3,exchange,14zRXp2AZ8yxXzPXJ5gAirmgAxJdUZiVut
4,mining pool,1F1xcRt8H8Wa623KqmkEontwAAVqDSAWCV


In [4]:
category_label = list(set(select_category_addr['Category']))
category_per_dict = dict() # 카테고리별 RootAddress들의 모음 list를 담은 dict
for cl in category_label:
    cl_per_addr = set(select_category_addr[select_category_addr['Category'] == cl]['RootAddress'])
    category_per_dict[cl] = cl_per_addr

In [5]:
category_list = list(category_per_dict.keys())

In [6]:
def apply_multi_input(addr_id):
    addres_for_cluster = set()
    
    cur.execute('''SELECT tx FROM TxIn WHERE addr = ?''', (addr_id,))
    for row in cur:
        addres_for_cluster.add(row[0])
        
    return addres_for_cluster

In [7]:
category_multi_input = dict()

for cpd in category_per_dict:
    count = 0
    for addr in category_per_dict[cpd]:
        cur.execute('''select id from AddrID where addr = ?''', (addr,))
        category_multi_input[str(cpd) + str(count)] = apply_multi_input(cur.fetchone()[0])
        count += 1

In [8]:
category_multi_input # 카테고리의 클러스터의 주소들의 인덱스 값

{'gambling0': {28987546, 29552101},
 'gambling1': {151495611},
 'gambling2': {20626479},
 'gambling3': {32988282,
  33062319,
  33062320,
  33219781,
  33264330,
  33299848,
  33351438,
  33352285,
  33415520,
  33422097,
  33428308,
  33431020,
  33436602,
  33437627,
  33446352,
  33448064,
  33451096,
  33501577,
  33516495,
  33517364,
  33569625,
  33571603,
  33656367,
  33684599,
  33700076,
  33704473,
  33762973,
  33763364,
  33788824,
  33789066,
  33791479,
  33837274,
  33838661,
  33873640,
  33885190,
  33926566,
  33927727,
  33928040,
  33941682,
  33973709,
  33981540,
  33987092,
  34003232,
  34018748,
  34031707,
  34032157,
  34050354,
  34088868,
  34164091,
  34168792,
  34188186,
  34188187,
  34231075,
  34231531,
  34320288,
  34320624,
  34320866,
  34321478,
  34321773,
  34321775,
  34322498,
  34322934,
  34341483,
  34404289,
  34499564,
  34500074,
  34529144,
  34641132,
  34662604,
  34672354,
  34722671,
  34722681,
  34732368,
  34736584,
  34753121

### connected component

In [9]:
import copy

apply_multiinput = copy.deepcopy(category_multi_input)

In [10]:
def searched_addresses(tx_id):
    address_set = set()
    
    cur.execute('''SELECT addr FROM TxIn WHERE tx = ?''', (tx_id,))
    for row in cur:
        address_set.add(row[0])
            
    return address_set

In [11]:
def get_addresses_same_cluster(addr_id):
    result = set()
    try:
        use_addr_tx = apply_multi_input(addr_id)
        for uat in use_addr_tx:
            result.update(searched_addresses(uat))
    except RuntimeError:
            return {addr_id}
        
    return result

In [ ]:
category_clusters = dict()
for i in range(10):
    for aml in apply_multiinput:
        result = set()
        target = apply_multiinput[aml]
        while len(target) != 0:
            address = target.pop()
            target.add(address)
            cluster = get_addresses_same_cluster(address)
            inter_set = target.intersection(cluster)
            result.update(cluster)
            target.difference_update(inter_set)
        category_clusters[aml] = result

In [ ]:
for cl in category_list:
    count = list()
    for cm in category_clusters:
        if cl in cm:
            count.append(len(category_clusters[cm]))
    print(cl + " - Min : " + str(min(count)) + " Max : " + str(max(count)) + " Avg : " + str(np.mean(count)) + " Total : " + str(len(count)))

### ransomware28번째 추가를 해야하며, 국내 거래소는 변경해야함

In [110]:
cransom = pd.read_csv('./crawler_ransomware.csv')
cransom_set = set(cransom['Bitcoin Address'].values)

In [111]:
# ransomware 수집했었던 것을 1개로 가져와서 추가
five_category_clusters['ransomware28'] = cransom_set